# OCR

In [1]:
import pytesseract
from pdf2image import convert_from_path

: 

In [2]:
# pdf_path = '../data/patente.pdf'
# pages = convert_from_path(pdf_path)

# # Aplique OCR em cada página
# full_text = ""

# for page_number, page in enumerate(pages):
#     # Converta a página da imagem para texto
#     text = pytesseract.image_to_string(page)
#     full_text += text

# with open('../data/patente.txt', 'w') as file:
#     file.write(full_text)

# Clean Data

In [3]:
import re

def split_document(document):
    # Usar expressão regular para dividir com base em duas ou mais quebras de linha consecutivas (\n\n ou mais)
    return re.split(r'\n\s*\n', document)

# LLM

In [4]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [5]:
file_path = "../data/patente.txt"

with open(file_path, 'r') as file:
    data = file.read()


In [6]:
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    # "You are a knowledgeable assistant. Given the following document related to glass, your task is to analyze the document and extract the chemical composition the given glass, detailing the percentage of each element, and listing the key properties associated with the glass. If the document does not contain relevant information regarding the chemical composition or properties, return 'not available'. Here are the contents of the document:\n\n----\n\n{document}."
    "You are a knowledgeable assistant. Given the following document related to glass, your task is to analyze the document and extract the chemical composition the given glass, detailing the percentage of each element, and listing the key properties associated with the glass. If there is no relevant information, do not return a generic aswer, just say 'there is no information'. Here are the contents of the document:\n\n----\n\n{document}."
  ),
  ("human", "{input}")
])

llm = ChatOllama(
  model="llama3.1",
  temperature=0.8,
)

chain = prompt | llm



In [7]:
chunks = split_document(data)

In [ ]:
chunk = chunks[17:50]
print(chunk)

In [19]:
# Processar cada parte separadamente e coletar as respostas
# for i, chunk in enumerate(chunks):
ai_msg = chain.invoke({
    "document": chunk,  # Passa o pedaço do documento
    "input": "Respond to me in JSON format. List the chemical compositions of the glass mentioned in the document along with the percentage of each element, and the key properties associated with each glass type. "
})


In [ ]:
print(ai_msg.content)